## qwen MOE trace and analysis module

In [ ]:
# import datasets
from datasets import load_dataset
data_id = "wikimedia/wikipedia"
sub_set_id = "20231101.zh-classical"
split = "train"
dataset = load_dataset(data_id, sub_set_id, split=split)

In [1]:
from v1 import Qwen3MoeDecoderLayerTimed


def modify_qwen3_moe_block(type__: str):
    from transformers.models.qwen3_moe import modeling_qwen3_moe as qmoe
    if type__ == "v1":
        qmoe.Qwen3MoeDecoderLayer = Qwen3MoeDecoderLayerTimed
    else:
        pass


modify_qwen3_moe_block("v1")

/home/hwk/data/code/qwen_moe/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-30B-A3B-Instruct-2507"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype="auto",
    device_map="auto"
)
model.eval()

Loading checkpoint shards: 100%|██████████| 16/16 [00:19<00:00,  1.20s/it]


Qwen3MoeForCausalLM(
  (model): Qwen3MoeModel(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-47): 48 x Qwen3MoeDecoderLayerTimed(
        (self_attn): Qwen3MoeAttention(
          (q_proj): Linear(in_features=2048, out_features=4096, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=4096, out_features=2048, bias=False)
          (q_norm): Qwen3MoeRMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3MoeRMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MoeSparseMoeBlockV1(
          (gate): Linear(in_features=2048, out_features=128, bias=False)
          (experts): ModuleList(
            (0-127): 128 x Qwen3MoeMLP(
              (gate_proj): Linear(in_features=2048, out_features=768, bias=False)
              (up_proj): Linear(in_features=2048, out_features=768, bias=False)
              (down_proj):

In [3]:
text_list = ["explain the qwen"]
tokenizer.padding_side = "left"
input_001 = tokenizer(text_list, return_tensors="pt", padding=True, truncation=True).to(model.device)

input_001


{'input_ids': tensor([[94344,   279,  2804, 16948]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1]], device='cuda:0')}

In [ ]:
import torch
import v1
# warm up
with torch.no_grad():
    model_output = model(**input_001)
v1.reset_timers()

from torch.profiler import profile, ProfilerActivity
with profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
    record_shapes=True, with_stack=True, profile_memory=True
) as prof:
    with torch.no_grad():
        _ = model(**input_001)
v1.show_res()
prof.export_chrome_trace("trace/trace_qwen3_moe_v1.json")

per-layer ms:
L00	attn=0.000	mlp=0.000	expert=0.000	softmax=0.000	gating=0.000
L01	attn=1.003	mlp=9.010	expert=8.522	softmax=0.158	gating=0.118
L02	attn=0.989	mlp=10.608	expert=10.111	softmax=0.157	gating=0.122
L03	attn=0.987	mlp=10.204	expert=9.706	softmax=0.158	gating=0.119
L04	attn=1.041	mlp=10.163	expert=9.648	softmax=0.166	gating=0.131
L05	attn=1.076	mlp=11.434	expert=10.794	softmax=0.257	gating=0.129
L06	attn=1.092	mlp=7.775	expert=7.257	softmax=0.168	gating=0.128
L07	attn=1.012	mlp=9.890	expert=9.375	softmax=0.165	gating=0.125
L08	attn=1.086	mlp=9.427	expert=8.940	softmax=0.159	gating=0.116
L09	attn=1.020	mlp=9.089	expert=8.592	softmax=0.160	gating=0.126
L10	attn=1.019	mlp=11.264	expert=10.759	softmax=0.167	gating=0.119
L11	attn=1.086	mlp=10.548	expert=9.996	softmax=0.171	gating=0.131
L12	attn=1.213	mlp=11.337	expert=10.773	softmax=0.177	gating=0.140
L13	attn=1.198	mlp=10.109	expert=9.580	softmax=0.173	gating=0.130
L14	attn=1.017	mlp=10.485	expert=9.984	softmax=0.161	gating=0.12